In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics #==8.2.103 -q
!yolo settings sync=False

import ultralytics
ultralytics.checks()

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 38.3/112.6 GB disk)


In [ ]:
import os
import glob
import time
import csv
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO


GDrive_DIR = "/content/drive/MyDrive/Phd Projects/DPC-4 Works"
os.chdir(GDrive_DIR)

# Models
MODEL_DIR = "models"
MODEL_PATHS = {
    "yolov8l_base": os.path.join(MODEL_DIR, "yolov8l.pt"),
    "yolo11l_base": os.path.join(MODEL_DIR, "yolo11l.pt"),
    "yolo11l_trained": os.path.join(MODEL_DIR, "yolo11l_251019.pt"),
}

# Image & video inputs
IMAGES_DIR = "data/images"

# Dataset for mAP evaluation
DATA_YAML = "data/data.yaml"
VAL_SPLIT = "val"                     # "val" or "test"

# Inference settings
DEVICE = 0            # 0 for first GPU, or "cpu"
IMG_SIZE = 640
CONF_THRES = 0.7
IOU_THRES = 0.45
MAX_VIDEO_FRAMES = 300   # limit for timing; set None to use full video

# Output root
OUTPUT_ROOT = "outputs_compare"
IMAGE_PRED_DIR = os.path.join(OUTPUT_ROOT, "images")
VIDEO_PRED_DIR = os.path.join(OUTPUT_ROOT, "videos")
REPORTS_DIR = os.path.join(OUTPUT_ROOT, "reports")
PLOTS_DIR = os.path.join(OUTPUT_ROOT, "plots")

for d in [IMAGE_PRED_DIR, VIDEO_PRED_DIR, REPORTS_DIR, PLOTS_DIR]:
    os.makedirs(d, exist_ok=True)

In [ ]:
# ============================================================
# UTILS
# ============================================================

def ensure_dir(path: str):
    Path(path).mkdir(parents=True, exist_ok=True)


def load_models(model_paths: dict):
    """Load YOLO models from given paths."""
    models = {}
    for name, path in model_paths.items():
        print(f"[INFO] Loading model '{name}' from {path} ...")
        if not os.path.isfile(path):
            raise FileNotFoundError(f"Model file not found: {path}")
        models[name] = YOLO(path)
    return models


def get_image_paths(images_dir: str):
    exts = ["*.jpg", "*.jpeg", "*.png", "*.bmp"]
    files = []
    for ext in exts:
        files.extend(glob.glob(os.path.join(images_dir, ext)))
    files = sorted(files)
    print(f"[INFO] Found {len(files)} images in {images_dir}")
    return files

In [ ]:
# ============================================================
# 1) IMAGE INFERENCE BENCHMARK
# ============================================================

def benchmark_on_images(models: dict, images_dir: str):
    image_paths = get_image_paths(images_dir)
    if not image_paths:
        print("[WARN] No images found for image benchmark. Skipping.")
        return

    per_image_csv = os.path.join(REPORTS_DIR, "image_per_image_stats.csv")
    summary_csv = os.path.join(REPORTS_DIR, "image_summary_stats.csv")

    with open(per_image_csv, "w", newline="") as f_per, open(summary_csv, "w", newline="") as f_sum:
        per_writer = csv.writer(f_per)
        sum_writer = csv.writer(f_sum)

        per_writer.writerow([
            "model", "image_name", "latency_sec",
            "num_detections", "mean_confidence"
        ])
        sum_writer.writerow([
            "model", "num_images",
            "avg_latency_sec", "fps",
            "avg_num_detections", "avg_mean_confidence"
        ])

        for model_name, model in models.items():
            print(f"\n[INFO] IMAGE benchmark for model: {model_name}")
            model_out_dir = os.path.join(IMAGE_PRED_DIR, model_name)
            ensure_dir(model_out_dir)

            latencies = []
            num_dets_list = []
            mean_conf_list = []

            for img_path in image_paths:
                img_name = os.path.basename(img_path)

                t0 = time.perf_counter()
                results = model(
                    source=img_path,
                    imgsz=IMG_SIZE,
                    conf=CONF_THRES,
                    iou=IOU_THRES,
                    device=DEVICE,
                    verbose=False,
                    save=False
                )
                t1 = time.perf_counter()
                dt = t1 - t0
                latencies.append(dt)

                res = results[0]
                boxes = res.boxes
                num_dets = 0
                mean_conf = 0.0

                if boxes is not None and len(boxes) > 0:
                    num_dets = len(boxes)
                    try:
                        mean_conf = float(boxes.conf.mean())
                    except Exception:
                        mean_conf = 0.0

                num_dets_list.append(num_dets)
                mean_conf_list.append(mean_conf)

                # Save prediction image
                plotted = res.plot()
                out_path = os.path.join(model_out_dir, img_name)
                cv2.imwrite(out_path, plotted)

                per_writer.writerow([
                    model_name, img_name, f"{dt:.6f}",
                    num_dets, f"{mean_conf:.4f}"
                ])

            if latencies:
                avg_latency = sum(latencies) / len(latencies)
                fps = 1.0 / avg_latency if avg_latency > 0 else 0.0
                avg_dets = sum(num_dets_list) / len(num_dets_list)
                avg_conf = sum(mean_conf_list) / len(mean_conf_list)

                sum_writer.writerow([
                    model_name,
                    len(image_paths),
                    f"{avg_latency:.6f}",
                    f"{fps:.2f}",
                    f"{avg_dets:.3f}",
                    f"{avg_conf:.4f}",
                ])

                print(
                    f"[IMAGE] {model_name}: "
                    f"avg latency = {avg_latency:.4f}s, FPS = {fps:.2f}, "
                    f"avg dets/img = {avg_dets:.2f}, avg conf = {avg_conf:.3f}"
                )

In [ ]:
# ============================================================
# 3) ACCURACY EVALUATION (.val) + PER-CLASS AP
# ============================================================

def evaluate_accuracy(models: dict, data_yaml: str, split: str = "val"):
    if data_yaml is None or not os.path.isfile(data_yaml):
        print("[WARN] DATA_YAML not provided or file not found. Skipping accuracy evaluation.")
        return

    metrics_csv = os.path.join(REPORTS_DIR, "accuracy_metrics.csv")
    per_class_csv = os.path.join(REPORTS_DIR, "per_class_ap.csv")

    with open(metrics_csv, "w", newline="") as f_global, \
         open(per_class_csv, "w", newline="") as f_pc:

        global_writer = csv.writer(f_global)
        pc_writer = csv.writer(f_pc)

        global_writer.writerow([
            "model", "dataset_split",
            "map_50_95", "map_50",
            "precision", "recall",
            "speed_preprocess_ms", "speed_inference_ms", "speed_postprocess_ms"
        ])

        pc_writer.writerow([
            "model", "class_id", "class_name",
            "map_50_95", "map_50"   # per-class mAP@0.5:0.95 and mAP@0.5
        ])

        for model_name, model in models.items():
            print(f"\n[INFO] VAL evaluation for model: {model_name}")
            metrics = model.val(
                data=data_yaml,
                split=split,
                imgsz=IMG_SIZE,
                device=DEVICE,
                conf=CONF_THRES,
                iou=IOU_THRES,
                verbose=False
            )

            # ---- Global metrics ----
            try:
                map_50_95 = float(metrics.box.map)     # mAP@0.5:0.95
                map_50 = float(metrics.box.map50)      # mAP@0.5
                precision = float(metrics.box.mp)      # mean precision
                recall = float(metrics.box.mr)         # mean recall

                sp = metrics.speed
                sp_p = float(sp.get("preprocess", 0.0))
                sp_i = float(sp.get("inference", 0.0))
                sp_o = float(sp.get("postprocess", 0.0))
            except Exception as e:
                print(f"[WARN] Could not parse global metrics for {model_name}: {e}")
                map_50_95 = map_50 = precision = recall = 0.0
                sp_p = sp_i = sp_o = 0.0

            global_writer.writerow([
                model_name, split,
                f"{map_50_95:.4f}",
                f"{map_50:.4f}",
                f"{precision:.4f}",
                f"{recall:.4f}",
                f"{sp_p:.3f}", f"{sp_i:.3f}", f"{sp_o:.3f}"
            ])

            print(
                f"[VAL] {model_name}: "
                f"mAP50-95={map_50_95:.4f}, mAP50={map_50:.4f}, "
                f"P={precision:.4f}, R={recall:.4f}"
            )

            # ---- Per-class AP ----
            try:
                # maps: array-like, mAP@0.5:0.95 for each class
                maps = metrics.box.maps  # shape [num_classes]
                # Some versions also expose map50s
                map50s = getattr(metrics.box, "map50s", None)

                # Class names: try metrics.names, fallback to model.names
                if hasattr(metrics, "names") and metrics.names is not None:
                    names = metrics.names
                else:
                    names = model.names

                # names may be dict or list
                if isinstance(names, dict):
                    id_to_name = names
                else:
                    id_to_name = {i: n for i, n in enumerate(names)}

                num_classes = len(maps)

                for cid in range(num_classes):
                    cls_name = id_to_name.get(cid, f"class_{cid}")
                    ap_50_95 = float(maps[cid])
                    if map50s is not None:
                        ap_50 = float(map50s[cid])
                    else:
                        # Fallback: use global map50 if per-class not available
                        ap_50 = map_50

                    pc_writer.writerow([
                        model_name, cid, cls_name,
                        f"{ap_50_95:.44f}",
                        f"{ap_50:.4f}"
                    ])

            except Exception as e:
                print(f"[WARN] Could not parse per-class AP for {model_name}: {e}")

In [ ]:
# ============================================================
# 4) PLOTTING
# ============================================================

def plot_fps_vs_map(
    accuracy_csv: str,
    video_summary_csv: str,
    out_path: str
):
    """
    Scatter plot: FPS (x-axis) vs mAP50-95 (y-axis)
    One point per model.
    """
    if not (os.path.isfile(accuracy_csv) and os.path.isfile(video_summary_csv)):
        print("[WARN] FPS vs mAP plot skipped (CSV missing).")
        return

    # Read mAP
    map_dict = {}  # model -> mAP50-95
    with open(accuracy_csv, "r") as f:
        reader = csv.DictReader(f)
        for row in reader:
            model = row["model"]
            map_50_95 = float(row["map_50_95"])
            map_dict[model] = map_50_95

    # Read FPS (from video benchmark)
    fps_dict = {}  # model -> FPS
    with open(video_summary_csv, "r") as f:
        reader = csv.DictReader(f)
        for row in reader:
            model = row["model"]
            fps = float(row["fps"])
            fps_dict[model] = fps

    models_common = sorted(set(map_dict.keys()) & set(fps_dict.keys()))
    if not models_common:
        print("[WARN] No overlapping models for FPS vs mAP plot.")
        return

    xs = [fps_dict[m] for m in models_common]
    ys = [map_dict[m] for m in models_common]

    plt.figure()
    plt.scatter(xs, ys)
    for x, y, label in zip(xs, ys, models_common):
        plt.text(x, y, label, fontsize=9, ha="right", va="bottom")

    plt.xlabel("Video FPS (higher is better)")
    plt.ylabel("mAP@0.5:0.95 (higher is better)")
    plt.title("Speed vs Accuracy: YOLO Models Comparison")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(out_path, dpi=300)
    plt.close()
    print(f"[PLOT] Saved FPS vs mAP plot to: {out_path}")


def plot_per_class_ap(per_class_csv: str, out_dir: str):
    """
    For each model, plot per-class mAP50-95 as a bar chart.
    """
    if not os.path.isfile(per_class_csv):
        print("[WARN] per_class_ap.csv not found. Skipping per-class AP plots.")
        return

    # Load data by model
    model_to_records = {}  # model -> list of (class_name, map_50_95)
    with open(per_class_csv, "r") as f:
        reader = csv.DictReader(f)
        for row in reader:
            model = row["model"]
            cls_name = row["class_name"]
            ap_50_95 = float(row["map_50_95"])
            model_to_records.setdefault(model, []).append((cls_name, ap_50_95))

    for model, records in model_to_records.items():
        # Sort by AP descending for nicer plots
        records_sorted = sorted(records, key=lambda x: x[1], reverse=True)
        class_names = [r[0] for r in records_sorted]
        ap_values = [r[1] for r in records_sorted]

        plt.figure(figsize=(max(6, 0.4 * len(class_names)), 4))
        x = range(len(class_names))
        plt.bar(x, ap_values)
        plt.xticks(x, class_names, rotation=45, ha="right")
        plt.ylabel("mAP@0.5:0.95")
        plt.ylim(0.0, 1.0)
        plt.title(f"Per-class AP (mAP@0.5:0.95) - {model}")
        plt.tight_layout()

        out_path = os.path.join(out_dir, f"per_class_ap_{model}.png")
        plt.savefig(out_path, dpi=300)
        plt.close()
        print(f"[PLOT] Saved per-class AP plot for {model} to: {out_path}")


In [ ]:
# ============================================================
# MAIN
# ============================================================

def main():
    print("[INFO] Starting YOLO model comparison...")

    models = load_models(MODEL_PATHS)

    # 1) Images
    benchmark_on_images(models, IMAGES_DIR)

    # 3) Accuracy (val/test)
    evaluate_accuracy(models, DATA_YAML, split=VAL_SPLIT)

    # 4) Plots
    accuracy_csv = os.path.join(REPORTS_DIR, "accuracy_metrics.csv")
    video_csv = os.path.join(REPORTS_DIR, "video_summary_stats.csv")
    per_class_csv = os.path.join(REPORTS_DIR, "per_class_ap.csv")

    # FPS vs mAP
    fps_map_plot_path = os.path.join(PLOTS_DIR, "fps_vs_map50_95.png")
    plot_fps_vs_map(accuracy_csv, video_csv, fps_map_plot_path)

    # Per-class AP
    plot_per_class_ap(per_class_csv, PLOTS_DIR)

    print("\n[INFO] All benchmarks and plots finished.")
    print(f"[INFO] Check outputs under: {OUTPUT_ROOT}/")


if __name__ == "__main__":
    main()

[INFO] Starting YOLO model comparison...
[INFO] Loading model 'yolov8l_base' from models/yolov8l.pt ...
[INFO] Loading model 'yolo11l_base' from models/yolo11l.pt ...
[INFO] Loading model 'yolo11l_trained' from models/yolo11l_251019.pt ...
[INFO] Found 23 images in data/images

[INFO] IMAGE benchmark for model: yolov8l_base
[IMAGE] yolov8l_base: avg latency = 0.1192s, FPS = 8.39, avg dets/img = 14.13, avg conf = 0.758

[INFO] IMAGE benchmark for model: yolo11l_base
[IMAGE] yolo11l_base: avg latency = 0.0895s, FPS = 11.18, avg dets/img = 13.43, avg conf = 0.738

[INFO] IMAGE benchmark for model: yolo11l_trained
[IMAGE] yolo11l_trained: avg latency = 0.0854s, FPS = 11.70, avg dets/img = 7.04, avg conf = 0.729
[WARN] DATA_YAML not provided or file not found. Skipping accuracy evaluation.
[WARN] FPS vs mAP plot skipped (CSV missing).
[WARN] per_class_ap.csv not found. Skipping per-class AP plots.

[INFO] All benchmarks and plots finished.
[INFO] Check outputs under: outputs_compare/
